# Deep Learning &mdash; Assignment 6

Sixth assignment for the 2023 Deep Learning course (NWI-IMC070) of the Radboud University.

-----

**Names:**

**Group:**

-----

**Instructions:**
* Fill in your names and the name of your group.
* Answer the questions and complete the code where necessary.
* Keep your answers brief, one or two sentences is usually enough.
* Re-run the whole notebook before you submit your work.
* Save the notebook as a PDF and submit that in Brightspace together with the `.ipynb` notebook file.
* The easiest way to make a PDF of your notebook is via File > Print Preview and then use your browser's print option to print to PDF.

## Objectives

In this assignment you will
1. Construct a PyTorch `DataSet`.
1. Train and modify a transformer network.
1. Learn how to use a model based on the PyTorch documentation.
1. Experiment with a translation dataset.

## Required software

If you haven't done so already, you will need to install the following additional libraries:
* `torch` for PyTorch,

All libraries can be installed with `pip install`.

In [ ]:
%matplotlib inline
import math
import time
from random import Random
from typing import (List, Optional)
import numpy as np
import torch
from torch import nn
from torch.utils.data import (IterableDataset, DataLoader)
import matplotlib.pyplot as plt
from IPython import display

# fix the seed, so outputs are exactly reproducible
torch.manual_seed(12345);

# Use the GPU if available
def detect_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    else:
        return torch.device("cpu")
device = detect_device()

## 6.1 Learning to calculate (5 points)

In this assignment we are going to train a neural network to do mathematics.
When communicating between humans, mathematics is expressed with words and formulas.
The simplest of these are formulas with a numeric answer. For example, we might ask what is `100+50`, to which the answer is `150`.

To teach a computer how to do this task, we are going to need a dataset.

Below is a function that generates a random formula. Study it, and see if you understand its parameters.

In [ ]:
def random_integer(length: int, signed: bool = True, rng: Random = Random()):
    max = int(math.pow(10, length))
    min = -max if signed else 0
    return rng.randint(min, max)

def random_formula(complexity: int, signed: bool = True, rng: Random = Random()):
    """
    Generate a random formula of the form "a+b" or "a-b".
    complexity is the maximum number of digits in the numbers.
    """
    a = random_integer(complexity, signed, rng)
    b = random_integer(complexity, False, rng)
    is_addition = not signed or rng.choice([False, True])
    if is_addition:
        return (f"{a}+{b}", str(a + b))
    else:
        return (f"{a}-{b}", str(a - b))

In [ ]:
seed = 123456
random_formula(3, rng=Random(seed))

Note that the `rng` argument allows us to reproduce the same random numbers, which you can verify by running the code below multiple times. But if you change the seed to `None` then the random generator is initialized differently each time.

In [ ]:
def random_formulas(complexity, signed, count, seed):
    """
    Iterator that yields the given count of random formulas
    """
    rng = Random(seed)
    for i in range(count):
        yield random_formula(complexity, signed, rng=rng)

for q, a in random_formulas(3, True, 5, seed):
    print(f'{q} = {a}')

We are going to treat these expressions as sequences of tokens, where each character is a token. In addition we will need tokens to denote begin-of-sequence and end-of-sequence, as well as padding, for which we will use `'<bos>'`, `'<eos>'`, and `'<pad>'` respectively, as was done in the lecture.

### Creating a vocabulary

The vocabulary is the set of all possible tokens. 

To store the vocabulary, we will use a helper class:

In [ ]:
class Vocab:
    """A vocabulary."""
    def __init__(self, tokens: List[str], specials: Optional[List[str]] = None):
        """
        Construct a vocabulary given a list of tokens, and an optional list of special tokens
        """
        self.tokens = tokens
        if specials is not None:
            self.tokens.extend(specials)
        self.tokens.sort()
        self.token_to_idx = {token: idx for idx, token in enumerate(self.tokens)}

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, token):
        return self.lookup_index(token)

    def lookup_index(self, token: str) -> int:
        return self.token_to_idx[token]

    def lookup_indices(self, tokens: List[str]) -> List[int]:
        return [self.lookup_index(token) for token in tokens]

    def lookup_token(self, index: int) -> str:
        return self.tokens[index]

    def lookup_tokens(self, indices: List[int]) -> List[str]:
        return [self.lookup_token(index) for index in indices]

For this dataset we know beforehand what the vocabulary will be, so we can easily define it by hand.

**(a) What are the tokens in this dataset? Complete the code below.<span style="float:right"> (1 point)</span>**

In [ ]:
# TODO: fill in all possible tokens
vocab = Vocab([...], specials=[...])

We can print the vocabulary to double check that it makes sense:

In [ ]:
print('Vocabulary size:', len(vocab))
print('Vocabulary:', vocab.tokens)

We are now ready to tokenize and encode formula.

**(b) Complete the code below.<span style="float:right"> (1 point)</span>**

In [ ]:
def tokenize_and_encode(string: str, vocab=vocab) -> List[int]:
    # TODO: Tokenize the string and encode using the vocabulary.
    #       Include an end-of-string token (but not a begin-of-string token).

Let's test it on a random formula:

In [ ]:
q, a = random_formula(3, rng=Random(seed))
print('The question', q, 'and answer', a)
print('are encoded as', tokenize_and_encode(q), 'and', tokenize_and_encode(a))

# Check tokenize_and_encode
assert ''.join(vocab.lookup_tokens(tokenize_and_encode(q))) == q + '<eos>'
assert len(tokenize_and_encode(q)) == len(q) + 1
assert tokenize_and_encode("1+2") == vocab.lookup_indices(['1','+','2','<eos>'])

### Padding and trimming

Next, to be able to work with a whole dataset of these encoded sequences, they all need to be the same length.

**(c) Implement the function below that pads or trims the encoded token sequence as needed.<span style="float:right"> (1 point)</span>**

Hint: see [d2l section 10.5.3](http://d2l.ai/chapter_recurrent-modern/machine-translation-and-dataset.html#loading-sequences-of-fixed-length) for a very similar function.

In [ ]:
def pad_or_trim(tokens: List[int], target_length: int, vocab=vocab):
    # TODO return a padded or trimmed sequence

In [ ]:
# pad or trim q to get a sequence of 10 tokens
pad_or_trim(tokenize_and_encode(q), 10)

In [ ]:
# Check pad_or_trim
assert len(pad_or_trim([1,2,3,4,5],10)) == 10
assert len(pad_or_trim(list(range(20)),10)) == 10
assert vocab.lookup_tokens(pad_or_trim([1,2,3,4,5],10)[5:]) == ['<pad>','<pad>','<pad>','<pad>','<pad>'], \
       f"Incorrect padding tokens, found {vocab.to_tokens(pad_or_trim([1,2,3,4,5],10)[5:])}"

### Translating tokens

We can use `vocab.lookup_tokens` to convert the encoded token sequence back to something more readable:

In [ ]:
vocab.lookup_tokens(pad_or_trim(tokenize_and_encode(q), 10))

For convenience, we define the `decode_tokens` function to convert entire lists or tensors:

In [ ]:
def decode_tokens(t, vocab=vocab):
    # convert a list, tensor, or array of encoded tokens
    if isinstance(t, torch.Tensor):
        t = t.detach().cpu()
    else:
        t = torch.tensor(t)
    return np.asarray(vocab.lookup_tokens(list(t.flatten()))).reshape(*t.shape)

# convert all tokens at once
print(decode_tokens([pad_or_trim(tokenize_and_encode('513+1323'), 10),
                     pad_or_trim(tokenize_and_encode('412+42'), 10)]))

### Creating a dataset

The most convenient way to use a data generating function for training a neural network is to wrap it in a PyTorch `Dataset`. In this case, we will use an [IterableDataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.IterableDataset), which can be used as an iterator to walk over the samples in the dataset.

**(d) Complete the code below.<span style="float:right"> (1 point)</span>**

In [ ]:
class FormulaDataset(IterableDataset):
    def __init__(self, complexity, signed, count, seed=None, vocab=vocab):
        self.seed = seed
        self.complexity = complexity
        self.signed = signed
        self.count = count
        self.vocab = vocab
        self.max_question_length = 2 * complexity + 3
        self.max_answer_length = complexity + 2

    def __len__(self):
        return self.count

    # TODO: Complete the class definition.
    #       See the documentation for IterableDataset for examples.
    #       Make sure that the values yielded by the iterator are pairs of torch tensors.
    #       To create a repeatable dataset, always start with the same random seed.

**(e) Define a training set with 10000 formulas and a validation set with 5000 formulas, both with complexity 3.<span style="float:right"> (1 point)</span>**

Note: make sure that the training and validation set are different.

In [ ]:
complexity = 3
signed = True
# TODO: Your code here.
train_data = ...
val_data   = ...

As usual, we wrap each dataset in a `DataLoader` to create minibatches.

In [ ]:
# Define data loaders
batch_size = 125
data_loaders = {
    'train': torch.utils.data.DataLoader(train_data, batch_size=batch_size),
    'val':   torch.utils.data.DataLoader(val_data, batch_size=batch_size),
}

In [ ]:
# The code below checks that the datasets are defined correctly
train_loader = data_loaders['train']
val_loader = data_loaders['val']

from typing import Tuple
from typing_extensions import assert_type
for (name, loader), expected_size in zip(data_loaders.items(), [10000,5000]):
    first_batch = next(iter(loader))
    assert len(first_batch) == 2, \
           f"The {name} dataset should yield (question, answer) pairs when iterated over."
    assert torch.is_tensor(first_batch[0]), \
           f"The questions in the {name} dataset should be torch.tensors"
    assert tuple(first_batch[0].shape) == (batch_size, 2*complexity+3), \
           f"The questions in the {name} dataset should be of size (batch_size, max_question_length), i.e. {batch_size,2*complexity+3}, found {tuple(first_batch[0].shape)}"
    assert first_batch[0].dtype in [torch.int32,torch.int64], \
           f"The questions in the {name} dataset should be encoded as integers, found {first_batch[0].dtype}"
    assert torch.equal(next(iter(loader))[0], next(iter(loader))[0]), \
           f"The {name} dataset should be deterministic, it should produce the same data each time"
    assert all([len(batch[0]) == batch_size for batch in iter(loader)]), \
           f"Batches should all have the right size. Perhaps the batch size does not evenly divide the dataset size?"
    assert sum([len(batch[0]) for batch in iter(loader)]) == expected_size, \
           f"{name} dataset does not have the right size, expected {expected_size}, found {sum([len(batch[0]) for batch in iter(loader)])}."
assert not torch.equal(next(iter(train_loader))[0], next(iter(val_loader))[0]), \
       "The training data and validation data should not be the same"

## 6.2 Transformer inputs (10 points)

There is a detailed description of the transformer model in Bishop chapter 12, and in [d2l chapter 11](http://d2l.ai/chapter_attention-mechanisms-and-transformers/index.html). We will not use the code from the d2l book, and instead use [PyTorch's built-in Transformer layers](https://pytorch.org/docs/stable/nn.html#transformer-layers).

However, some details we still need to implement ourselves.

### Autoregressive inputs

We will be training the decoder of the transformer as an autoregressive model.

The code below takes a batch of data from the training set, and it generates a shifted version of the target values.

In [ ]:
def shift_targets(y, bos_token=vocab['<bos>']):
    """
    Shift a sequence of tokens by 1 position, and add `bos_token` at the start.
    """
    bos = torch.tensor(bos_token, dtype=y.dtype, device=y.device).expand(y.shape[0], 1)
    y_prev = torch.cat((bos, y[:, :-1]), axis=1)
    return y_prev

In [ ]:
x, y = next(iter(train_loader))
y_prev = shift_targets(y)

# print the first five samples
print(decode_tokens(y)[:5])
print(decode_tokens(y_prev)[:5])

**(a) Look at the values for the example above. What is `y_prev` used for during training of a transformer model?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

**(b) Why do some rows of `y_prev` end in `'<eos>'`, but not all? Is this a problem?<span style="float:right"> (1 point)</span>** 

TODO: Your answer here.

### Masks

Training a transformer uses masked self-attention, so we need some masks. Here are two functions that make these masks.

In [ ]:
def generate_square_subsequent_mask(size, device=device):
    """
    Mask that indicates that tokens at a position are not allowed to attend to
    tokens in subsequent positions.
    """
    mask = (torch.tril(torch.ones((size, size), device=device))) == 0
    return mask

def generate_padding_mask(tokens, padding_token):
    """
    Mask that indicates which tokens should be ignored because they are padding.
    """
    if not isinstance(tokens, torch.Tensor):
        tokens = torch.tensor(tokens)
    return tokens == padding_token

**(c) Generate a padding mask for a random encoded token string.<span style="float:right"> (1 point)</span>**

Hint: make sure that `tokens` is a torch.tensor.

In [ ]:
q, a = random_formula(3, rng=Random(seed))
# TODO: your code here
tokens = ...
padding_mask = ...
print(tokens)
print(decode_tokens(tokens))
print(padding_mask)

In [ ]:
# More tests
assert list(generate_padding_mask(torch.tensor(pad_or_trim(tokenize_and_encode("1+1"), 8)), vocab['<pad>'])) == [False]*4 + [True]*4, "Something is wrong with generate_padding_mask"

**(d) How will this mask be used by a transformer?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

The code below illustrates what the output of `generate_square_subsequent_mask` looks like.

In [ ]:
square_subsequent_mask = generate_square_subsequent_mask(y.shape[1])

print(square_subsequent_mask.shape)
print(square_subsequent_mask)

**(e) How and why should this mask be used? State your answer in terms of `x`,  `y` and/or `y_prev`.<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

**(f) Give an example where it could make sense to use a different mask in a transformer network, instead of the `square_subsequent_mask`?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

### Embedding

Our discrete vocabulary is not suitable as the input for a transformer. We need an embedding function to map our input vocabulary to a continuous, high-dimensional space.

We will use the `torch.nn.Embedding` class to for this. As you can read in the [documentation](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html#torch.nn.Embedding), this class maps each token in our vocabulary to a specific point in embedding space, its embedding vector. We will use this embedding vector as the input features for the next layer of our model.

The parameters of the embedding are trainable: the embedding vector of each token is optimized along with the rest of the network.

**(g) Define an embedding that maps our vocabulary to a 5-dimensional space.<span style="float:right"> (1 point)</span>**

In [ ]:
# TODO: Your code here.
embedding = ...
print(embedding)

Let's apply the embedding to some sequences from our training set.

In [ ]:
# take the first batch
x, y = next(iter(train_loader))
# take three samples
x = x[:3]
# print the shapes
print(x)
print(embedding(x))
print(x.shape)
print(embedding(x).shape)

**(h) Explain the output shape.<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

The size of the embedding vectors, or the dimensionality of the embedding space, does not depend on the number of tokens in our vocabulary. We are free to choose an embedding size that fits our problem.

For example, let's try an embedding with 2 dimensions, and plot the initial embedding for the tokens in our vocabulary.

**(i) Create an embedding with 2 dimensions and plot the embedding for all tokens.<span style="float:right"> (no points)</span>**

In [ ]:
# TODO: Your code here.
embedding = ...

# embed all tokens of our vocabulary
x = torch.arange(len(vocab))
emb = embedding(x).detach().cpu().numpy()

plt.scatter(emb[:, 0], emb[:, 1]);
for i, token in enumerate(vocab.tokens):
    plt.annotate(token, (emb[i,0]+0.04, emb[i,1]))

As always, we need to balance the complexity of our networks: a larger embedding will increase the number of parameters in our model, but increase the risk of overfitting.

**(j) Would this 2-dimensional embedding space be large enough for our problem?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

Instead of using an embedding, we could also use a simple one-hot encoding to map the words in the vocabulary to feature vectors. However, practical applications of natural language processing never do this. Why not?

**(k) Explain the practical advantage of embeddings over one-hot encoding.<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

## 6.3 `torch.nn.Transformer` (8 points)

<div style="float: left"><a href="https://cs.ru.nl/~gvtulder/vaswani-fig-1-highlight.png"><img src="https://cs.ru.nl/~gvtulder/vaswani-fig-1-highlight.png" width="300"></a></div>

We now have all required inputs for our transformer.

Consult the documentation for the [`torch.nn.Transformer`](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html) class of PyTorch. This class implements a full Transformer as described in ["Attention Is All You Need"](https://arxiv.org/pdf/1706.03762.pdf), the paper that introduced this architecture.

The `Transformer` class implements the main part of the of the Transformer architecture, shown highlighted in the image on the left (see also Fig. 1 in "Attention Is All You Need").

For a given input sequence, it applies one or more encoder layers, followed by one or more decoder layers, to compute an output sequence that we can then process further.

Because the `Transformer` class takes care of most of the complicated parts of the model, we can concentrate providing the inputs and outputs: the grayed-out areas in the image.

Check out the parameters for the `Transformer` class and the inputs and outputs of its `forward` function.
<br style="clear: both">

**(a) Which parameter of the `Transformer` class should we base on our embedding?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

**(b) Given fixed input and output dimensions, which parameters of the `Transformer` can we use to change the complexity of our network?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

**(c) When using the `Transformer` class, where should we use the masks that we defined earlier?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

### Building a network

**(d) Complete the code for the TransformerNetwork.<span style="float:right"> (5 points)</span>**

Construct a network with the following architecture (see the image in the previous section for an overview):
1. An embedding layer that embeds the input tokens into a space of size `dim_hidden`.
2. A dropout layer (not shown in the image).
3. A [Transformer](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html) with the specified parameters (`dim_hidden`, `num_heads`, `num_layers`, `dim_feedforward`, and `dropout`).<br>Note: you will need to pass `batch_first=True`, to indicate that the first dimension runs over the batch and not over the sequence.
4. A final linear prediction layer that takes the output of the transformer to `dim_vocab` possible classes.

Don't worry about positional encoding for now, we will add that later.

The `forward` function should generate the appropriate masks and combine the layers defined in `__init__` to compute the output.

In [ ]:
class TransformerNetwork(torch.nn.Module):
    def __init__(self,
                 dim_vocab=len(vocab), padding_token=vocab['<pad>'],
                 num_layers=2, num_heads=4, dim_hidden=64, dim_feedforward=64,
                 dropout=0.01, positional_encoding=False):
        super().__init__()
        self.padding_token = padding_token
        # TODO: Your code here.
        self.embedding    = ...
        self.dropout      = ...
        self.transformer  = ...
        self.predict      = ...
        if positional_encoding:
            self.pos_encoding = ... # Fill this in later
        else:
            self.pos_encoding = torch.nn.Identity()

    def forward(self, src, tgt):
        # TODO: Your code here.
        # Combine self.embedding, self.dropout, self.transformer, self.predict

**(e) Try the transformer with an example batch.**

In [ ]:
net = TransformerNetwork(dim_feedforward=72)
x, y = next(iter(train_loader))
y_prev = shift_targets(y)

print('x.shape', x.shape)
print('y.shape', y.shape)
print('y_prev.shape', y_prev.shape)

y_pred = net(x, y_prev)
print('y_pred.shape', y_pred.shape)

# check the shape against what we expected
np.testing.assert_equal(list(y_pred.shape), [y.shape[0], y.shape[1], len(vocab)])

We can convert these predictions to tokens (but they're obviously random):

In [ ]:
print(decode_tokens(torch.argmax(y_pred, dim=2))[:5])

In [ ]:
# Check that the transformer is defined correctly
assert isinstance(net.embedding, torch.nn.Embedding)
assert isinstance(net.dropout, torch.nn.Dropout)
assert isinstance(net.transformer, torch.nn.Transformer)
assert isinstance(net.predict, torch.nn.Linear)
# Check parameters of transformer
assert net.transformer.d_model == 64
assert net.transformer.nhead == 4
assert net.transformer.batch_first == True
assert net.transformer.encoder.num_layers == 2
assert net.transformer.decoder.num_layers == 2
assert net.transformer.encoder.layers[0].linear1.out_features == 72
assert net.dropout.p == 0.01
assert net.transformer.encoder.layers[0].dropout.p == 0.01
# Check that the forward function behaves correctly
net.train(False)
assert torch.all(torch.isclose( \
            net(x, y_prev), \
            net(torch.cat((x,torch.tensor(vocab['<pad>']).expand(x.shape[0], 5)), axis=1), y_prev), atol=1e-5)), \
       "Adding padding to x should not affect the output of the network. Check src_key_padding_mask and memory_key_padding_mask. The former controls self attention to padding tokens in the encoder, the latter controls cross attention from decoder to encoder."
assert torch.all(torch.isclose( \
            net(x, y_prev), \
            net(x, torch.cat((y_prev,torch.tensor(vocab['<pad>']).expand(y.shape[0], 5)), axis=1))[:,:-5], atol=1e-5)), \
       "Adding padding to y should not affect the output of the network. Check tgt_key_padding_mask."
assert torch.all(torch.isclose( \
            net(x, y_prev)[:,:2], \
            net(x, y_prev[:,:2]), atol=1e-5)), \
       "The presence of later tokens in y should not affect the output for earlier tokens. Check tgt_mask."
assert torch.all(torch.isclose( \
            net(x, y_prev), \
            net(torch.flip(x, [1]), y_prev), atol=1e-5)), \
       "Order of x should not matter for a transformer network. Check src_mask."
assert not torch.all(torch.isclose( \
            net(x, torch.flip(y_prev, [1])), \
            torch.flip(net(x, y_prev), [1]), atol=1e-5)), \
       "Order of y should matter for a transformer network. Check tgt_mask."

## 6.4 Training (10 points)

### Training loop

We will base the training code on last week's code. A complication in computing the loss and accuracy are the padding tokens. So, before we work on the training loop itself, we need to update the `accuracy` function so it ingores these `<pad>` tokens. Let's do this in a generic way

**(a) Copy the `accuracy` function from last week, and add a parameter `ignore_index`. The tokens with `true_y == ignore_index` should be ignored.<span style="float:right"> (1 point)</span>**

Hint: you can select elements from a tensor with `some_tensor[include]` where `include` is a tensor of booleans.

In [ ]:
def accuracy(pred_y, true_y, ignore_index=None):
    # TODO: Your code here.
    # Hint: See assignment 3 or 4.

In [ ]:
# Test the accuracy function.
assert accuracy(torch.tensor([[1,0,0],[0.4,0.5,0.1],[0,1,0],[0.4,0.1,0.5]]), torch.tensor([0,1,2,2]), 1) == 2/3
assert accuracy(torch.tensor([[1,0,0],[0.4,0.5,0.1],[0,1,0],[0.4,0.1,0.5]]), torch.tensor([0,1,2,2]), 2) == 1
assert accuracy(torch.tensor([[1,0,0],[0.4,0.5,0.1],[0,1,0],[0.4,0.1,0.5]]), torch.tensor([0,1,2,2]), 3) == 3/4
assert accuracy(torch.tensor([[1,0,0],[0.4,0.5,0.1],[0,1,0],[0.4,0.1,0.5]]), torch.tensor([2,2,1,2]), 2) == 1

**(b) Write a training loop for the transformer model.<span style="float:right"> (4 points)</span>**

See last week's assignment for inspiration.
The code is mostly the same with the following changes:
 * The cross-entropy loss function and accuracy should ignore all `<pad>` tokens. (Use `ignore_index`, see the [documentation of CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).)
 * The network expects `y_prev` as an extra input.
 * The output of the network contains a batch of N samples, with maximum length L, and gives logits over C classes, so it has size (N,L,C). But `CrossEntropyLoss` and `accuracy` expect a tensor of size (N,C,L). You can use [torch.Tensor.transpose](https://pytorch.org/docs/stable/generated/torch.transpose.html) to change the output to the right shape.

In [ ]:
def train(net, data_loaders, num_epochs=100, lr=0.001, optimizer=torch.optim.Adam, device=device):
    """
    Train a network on the given data set.
    After every epoch compute validation loss and accuracy.
    """
    # TODO: Your code here.
    # Hint: See assignment 4.

### Experiment

**(c) Train a transformer network. Use 100 epochs with a learning rate of 0.001<span style="float:right"> (no points)</span>**

In [ ]:
# TODO: your answer here

**(d) Briefly discuss the results. Has the training converged? Is this a good calculator?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

**(e) Run the trained network with input `"123+123"` and `"321+321"`.<span style="float:right"> (1 point)</span>**

In [ ]:
def predict(net, q, a):
    # Run net to predict the output given the input `q` and y_prev based on `a`.
    # Return predicted y
    with torch.no_grad():
        # TODO: Your code here.
        pass

for src, tgt in [('123+123', '246'), ('321+321', '642')]:
    print(f'For {src}={tgt}')
    y_pred = predict(net, src, tgt)
    print('  y_pred[0]', y_pred[0])
    print('  encoded', torch.argmax(y_pred, dim=-1))
    print('  tokens', decode_tokens(torch.argmax(y_pred, dim=-1)))
    print()

**(f) Compare the predictions for the first element of y with the two different inputs. Can you explain what happens?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

**(g) Does the validation accuracy estimate how often the model is able to answers formulas correctly? Explain your answer.<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

**(h) If the forward function takes the shifted output `y_prev` as input, how can we use it if we don't know the output yet?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

## 6.5 Positional encoding (5 points)

We did not yet include positional encoding in the network.
PyTorch does not include such an encoder. So, here we define such a module ourselves:

In [ ]:
class PositionalEncoding(nn.Module):
    """Positional encoding."""
    def __init__(self, num_hiddens, max_len=100):
        super().__init__()
        # Create a long enough matrix of position encodings P
        positions = torch.arange(max_len, dtype=torch.float32)
        freqs = torch.pow(max_len, (1 + 2 * torch.arange(num_hiddens / 2)) / num_hiddens)
        X = positions[:,None] / freqs[None,:]
        self.P = torch.zeros((max_len, num_hiddens))
        self.P[:, 0::2] = torch.sin(X)
        self.P[:, 1::2] = torch.cos(X)

    def forward(self, X):
        return X + self.P[None, :X.shape[1], :].to(X.device)

**(a) Add positional encoding to the TransformerModel.<span style="float:right"> (point given in earlier question)</span>**

In [ ]:
# TODO: Modify the code in 6.3d.

**(b) Construct and train a network with positional encoding<span style="float:right"> (1 point)</span>**

In [ ]:
# TODO: your answer here
net_pos = ...

**(c) How does the performance of a model with positional encoding compare to a model without?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

**(d) Run the trained network with input `"123+123"` and `"321+321"`.<span style="float:right"> (no points)</span>**

In [ ]:
# TODO: Your code here.

**(e) Compare the predictions for the first element of y with what you found earlier. Can you explain what happens?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

**(f) Explain in your own words why positional encoding is used in transformer networks.<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

**(g) Look at the learning curve. Can you suggest a way to improve the model?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

**(h) Optional: if time permits, try to train an even better model**

## 6.6 Predicting for new samples (5 points)

Predicting an output given a new sample requires an appropriate search algorithm (see [d2l chapter 10.8](https://d2l.ai/chapter_recurrent-modern/beam-search.html)). Here, we will implement the simplest form: a greedy search algorithm that selects the token with the highest probability at each time step.

**(a) Describe this search strategy in pseudo-code.<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

**(b) Implement a greedy search function to predict a sequence using `net_pos`.<span style="float:right"> (2 points)</span>**

In [ ]:
def predict_greedy(net, src, length):
    # predict an output sequence of the given (maximum) length given input string src
    with torch.no_grad():
        # TODO: Your code here.
        pass

predicted_sequence = predict_greedy(net_pos, '123+123', 6)
print(decode_tokens(predicted_sequence))

**(c) Does this search strategy give a high-quality prediction? Why, or why not?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

**(d) What alternative search strategy could we use to improve the predictions? Why would this help?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

## 6.7 Discussion (4 points)

Last week, we looked at recurrent neural networks such as the LSTM. Both recurrent neural networks and transformers work with sequences, but in recent years the transformer has become more popular than the recurrent models.

**(a) An advantage of transformers over recurrent neural is that they can be faster to train. Why is that?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

**(b) Does this advantage also hold when predicting outputs for new sequences? Why, or why not?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

**(c) Why is positional encoding often used in transformers, but not in convolutional or recurrent neural networks?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

The structure of a recurrent neural network makes it very suitable for online predictions, such as real-time translation, because it only depends on prior inputs. You can design an architecture where the RNN produces an output token for every input token given to it, and it can produce that output without having to wait for the rest of the input.

Note: 'online' means producing outputs continuously as new input comes in, as opposed to collecting a full dataset and analyzing it afterwards, it has nothing to do with the internet.

**(d) How would a transformer work in an online application? Do you need to change the architecture?<span style="float:right"> (1 point)</span>**

TODO: Your answer here.

## The end

Well done! Please double check the instructions at the top before you submit your results.

*This assignment has 47 points.*
<span style="float:right;color:#aaa;font-size:10px;"> Version 4f237eb / 2024-10-08</span>